# RTBM Basic Tutorial

## Introduction

In this tutorial we will explain with examples how to operate with the RTBM package.
This package contains a python module with the following RTBM based objects:
- single RTBM 
- models based on layers (RTBM, linear, etc.)

These objects can be trained through multiple algorithms, however for the time being we will focus on genetic optimization.

## Importing the package

Before starting lets import this module:

In [ ]:
import rtbm

and some useful library for the data generation and ploting:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Example 1: RTBM density estimation

The rtbm.RTBM object allocates 1 RTBM with Nv and Nh, visible and hidden units.

In [ ]:
a = rtbm.RTBM(1,1) # allocates 1 RTBM with (Nv=1,Nh=1)
# This object is intialized in probability mode, you can change it 
# by setting the mode flag in the constructor which takes the rtbm.RTBM.Mode options.
#        Args:
#            visible_units: number of visible units
#            hidden_units: number of hidden units
#            mode: see Mode enumerator (Mode.Probability, Mode.LogProbability, Mode.Expectation)
#            init_max_param_bound: size of maximum parameters used in random initialization.
#            phase: number which multiplies w and bh

In [ ]:
# lest generate some simple gaussian data
n = 1000
data = (np.random.normal(5,10,n)).reshape(1,n)
plt.hist(data.T, bins=50, normed=True);

There are multiple minimization algorithms in `rtbm.minimizer`, here we will use the CMA.

In [ ]:
# and now we train with the CMA algorithm
from rtbm import minimizer
minim = minimizer.CMA()

In [ ]:
# and now we train with the logarithmic error function (see rtbm.costfunctions)
from rtbm.costfunctions import logarithmic
solution = minim.train(logarithmic(), a, data, tolfun=1e-4)

Then we check the output and compare to the input data:

In [ ]:
test_data = (np.linspace(-40,40, 100)).reshape(1, 100)
plt.hist(data.T, bins=50, normed=True)
plt.plot(test_data.flatten(), a(test_data).flatten(), 'o-')

## Example 2: Model regression

In this example we show how to allocate and train a RTBM model.

In [ ]:
# create a model with 3 layers of diagonal expectaion units
from rtbm import model, layers

M = model.Model()
M.add(layers.DiagExpectationUnitLayer(1,3,phase=1))
M.add(layers.DiagExpectationUnitLayer(3,3,phase=1))
M.add(layers.DiagExpectationUnitLayer(3,1,phase=1))

Then we generate the data with noise:

In [ ]:
def func(x,n):
    return 0.5*np.sin(1*x+0.1)+0.75*np.sin(0.25*x-0.3)+n*np.random.normal(0,0.2,x.shape[2])

X = np.linspace(0, 100, 500)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None],1).reshape((1,X.shape[1]))
plt.plot(X.flatten(), Y.flatten(),"ob-")

Finally we perform the regression with MSE cost:

In [ ]:
from rtbm import minimizer
from rtbm.costfunctions import mse

minim = minimizer.CMA()
minim.train(mse(), M, X, Y, tolfun=1e-4)